## Simple LLM Workflow

#### que. --> node(llm_qa) --> ans.

In [9]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

import os
key = os.getenv("GROQ_API_KEY")

In [10]:
model = ChatGroq(model="llama-3.3-70b-versatile", api_key=key)
model.invoke("hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.018977432, 'prompt_time': 0.001517526, 'queue_time': 0.052778724, 'total_time': 0.020494958}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c0d6248a-736b-491f-b821-cb6fbbaa6ad4-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [11]:
# create a state

class llmstate(TypedDict):

    que: str
    ans: str

In [25]:
# fn for the node

def llm_qa(state: llmstate) -> llmstate:
     
    # extract question from the state
    question = state["que"]

    # form a prompt
    prompt = f"Answer the following question{question}"

    #ask que to llm 
    answer = model.invoke(prompt)

    # updata answer to state
    state["ans"] = answer.content

    return state


In [26]:
#we will create a graph

graph = StateGraph(llmstate)

# add node 

graph.add_node('llm_qa', llm_qa)

# define edge
graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

# compile the graph
workflow = graph.compile()

In [27]:
#execute the graph

initial_state = {"que": "how fas is mars from earth"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'que': 'how fas is mars from earth', 'ans': 'The average distance from Earth to Mars is about 225 million kilometers (139.8 million miles). However, the distance between the two planets varies as both Earth and Mars follow elliptical orbits around the Sun.\n\nAt their closest, Mars and Earth can be as close as 56 million kilometers (34.8 million miles), which is known as "opposition" and occurs every 26 months. This is when Mars is on the opposite side of the Earth from the Sun.\n\nAt their farthest, Mars and Earth can be as far as 401 million kilometers (249.2 million miles) apart, which is known as "conjunction" and occurs when Mars is on the same side of the Sun as Earth.\n\nHere\'s a rough breakdown of the distances between Earth and Mars:\n\n* Closest (opposition): 56 million km (34.8 million miles)\n* Average: 225 million km (139.8 million miles)\n* Farthest (conjunction): 401 million km (249.2 million miles)\n\nIt\'s worth noting that the distance between Earth and Mars is cons

In [29]:
print(final_state["ans"])

The average distance from Earth to Mars is about 225 million kilometers (139.8 million miles). However, the distance between the two planets varies as both Earth and Mars follow elliptical orbits around the Sun.

At their closest, Mars and Earth can be as close as 56 million kilometers (34.8 million miles), which is known as "opposition" and occurs every 26 months. This is when Mars is on the opposite side of the Earth from the Sun.

At their farthest, Mars and Earth can be as far as 401 million kilometers (249.2 million miles) apart, which is known as "conjunction" and occurs when Mars is on the same side of the Sun as Earth.

Here's a rough breakdown of the distances between Earth and Mars:

* Closest (opposition): 56 million km (34.8 million miles)
* Average: 225 million km (139.8 million miles)
* Farthest (conjunction): 401 million km (249.2 million miles)

It's worth noting that the distance between Earth and Mars is constantly changing due to the elliptical shape of their orbits.